In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv('../data/data.tsv', sep='\t',error_bad_lines=False)

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363192 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363192 non-null  object 
 1   qid1          363192 non-null  object 
 2   qid2          363185 non-null  float64
 3   question1     363181 non-null  object 
 4   question2     363180 non-null  object 
 5   is_duplicate  363180 non-null  float64
dtypes: float64(2), object(4)
memory usage: 16.6+ MB


In [5]:
df = df.dropna(axis=0, subset=('question1','question2','is_duplicate' ))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


In [7]:
#lower casing
def lower_case(text):
    return text.lower()


#remove numbers
def remove_numbers(text):
    output = re.sub(r'\d+', '', text)
    return output

# remove punctuation
import string
def remove_punctuation(text):
    text_p = "".join([char for char in text if char not in string.punctuation])
    return text_p

#tokenize text

import nltk
nltk.download('punkt')
from nltk import word_tokenize

def tokenize(text):
    words = word_tokenize(text)
    return words

#remove stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stopwords(text):
    filtered_words = [word for word in text if word not in stop_words]
    return filtered_words

#stemming
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_words(text):
    stemmed = [stemmer.stem(word) for word in text]
    return stemmed

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
class Preprocessor():
    def __init__(self):
        pass
    
    def preprocess(self,df_column, steps):
        self.df_column = df_column
        self.steps = steps
    
        if 'lower_case' in self.steps:
            self.df_column = self.df_column.apply(lambda x: lower_case(x))
        
        
        if 'remove_numbers' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_numbers(x))
        
        if 'remove_punctuation' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_punctuation(x))           
        
        if 'tokenize' in self.steps:
            self.df_column = self.df_column.apply(lambda x: tokenize(x))
        
        if 'stopwords' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_stopwords(x))
        
        if 'stemming' in self.steps:
            self.df_column = self.df_column.apply(lambda x: stem_words(x))
            
        return self.df_column

In [10]:
steps = ['lower_case','remove_numbers','remove_punctuation',
        'tokenize','stopwords','stemming']
processor = Preprocessor()
df['question1'] = processor.preprocess(df['question1'] ,steps)

In [12]:
df['question2'] = processor.preprocess(df['question2'] ,steps)

In [13]:
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363177 entries, 0 to 363176
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         363177 non-null  int64  
 1   id            363177 non-null  object 
 2   qid1          363177 non-null  object 
 3   qid2          363177 non-null  float64
 4   question1     363177 non-null  object 
 5   question2     363177 non-null  object 
 6   is_duplicate  363177 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 19.4+ MB


In [14]:
df.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0
2,2,360472,364011,490273.0,"[caus, stool, color, chang, yellow]","[caus, stool, come, littl, ball]",0.0
3,3,150662,155721,7256.0,"[one, mbb]",[mbb],1.0
4,4,183004,279958,279959.0,"[find, power, outlet, laptop, melbourn, airport]","[would, second, airport, sydney, australia, ne...",0.0


In [15]:
def calc_tf(alist):
    
    tf_list = [alist.count(term)/len(alist) for term in alist]
    return tf_list

df['q2_tf'] = df['question2'].apply(lambda x: calc_tf(x))
df.head()

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_tf
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0,"[0.2, 0.2, 0.2, 0.2, 0.2]"
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0,"[0.5, 0.5]"
2,2,360472,364011,490273.0,"[caus, stool, color, chang, yellow]","[caus, stool, come, littl, ball]",0.0,"[0.2, 0.2, 0.2, 0.2, 0.2]"
3,3,150662,155721,7256.0,"[one, mbb]",[mbb],1.0,[1.0]
4,4,183004,279958,279959.0,"[find, power, outlet, laptop, melbourn, airport]","[would, second, airport, sydney, australia, ne...",0.0,"[0.08333333333333333, 0.08333333333333333, 0.0..."


In [19]:
q2vocab = df['question2'].tolist()
q2vocab = [item for sublist in q2vocab for item in sublist]
q2vocab = list(set(q2vocab))
len(q2vocab)

50127

In [21]:
def get_Doc_ids(vocabulary_list,dataframe,tokens_col_name, id_col_name):
    Doc_ids = {}      #initialise empty document frequency dictionary
    
    #for each word in vocabulary, create key:value pair, with an empty list as value
    for word in vocabulary_list:
        Doc_ids[word] = [] 
        
    #for each word in DF, add the id of question2 where it appears. Create a set to avoid duplicate ids
    for i in range(len(dataframe)):
        for word in dataframe.loc[i,tokens_col_name]:
            Doc_ids[word].append(dataframe[id_col_name][i])
    for word in Doc_ids.keys():
        Doc_ids[word] = list(set(Doc_ids[word]))
        #DF[word]  =len(DF[word])
        
    return Doc_ids

Doc_ids = get_Doc_ids(q2vocab,df,'question2','qid2')
list(Doc_ids.items())[:5]

[('einstein',
  [526339.0,
   50180.0,
   18447.0,
   35857.0,
   254996.0,
   370203.0,
   367646.0,
   225826.0,
   60963.0,
   369191.0,
   422954.0,
   428076.0,
   459318.0,
   464441.0,
   326714.0,
   503359.0,
   453703.0,
   52295.0,
   98378.0,
   173135.0,
   100944.0,
   179280.0,
   381527.0,
   21594.0,
   521324.0,
   143474.0,
   198774.0,
   165504.0,
   240786.0,
   141460.0,
   167063.0,
   461464.0,
   423576.0,
   349336.0,
   423071.0,
   476322.0,
   363171.0,
   363686.0,
   335531.0,
   5294.0,
   450735.0,
   25284.0,
   185544.0,
   221403.0,
   246517.0,
   33023.0,
   347910.0,
   535309.0,
   220429.0,
   317714.0,
   489774.0,
   87855.0,
   90415.0,
   88881.0,
   405812.0,
   524596.0,
   273219.0,
   487235.0,
   169811.0,
   160087.0,
   104792.0,
   361817.0,
   278368.0,
   441704.0,
   213355.0,
   393069.0,
   211825.0,
   455547.0,
   209796.0,
   262532.0,
   444298.0,
   315278.0,
   198550.0,
   306585.0,
   81824.0,
   322469.0,
   145325.0,


In [63]:
#create Document_frequency (IDF) dictionary
N = df['qid2'].nunique()
print(N)

IDF = {}
for word in Doc_ids.keys():
    IDF[word] = N/len(Doc_ids[word])+1
list(IDF.items())[:5]    

273121


[('einstein', 3002.3296703296705),
 ('header', 11875.826086956522),
 ('munchkin', 273122.0),
 ('hotelresort', 136561.5),
 ('expeci', 136561.5)]

In [64]:
df['q2_len'] = df['question2'].apply(lambda x: len(x) )
df.head(2)

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_tf,q2_len
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0,"[0.2, 0.2, 0.2, 0.2, 0.2]",5
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0,"[0.5, 0.5]",2


In [65]:
doc_words = dict(zip(df['qid2'],df['question2']))
tf_lists = {}
for word in q2vocab:
    word_tf = []
    for doc in Doc_ids[word]:
        tf = doc_words[doc].count(word)/len(doc_words[doc])
        word_tf.append(tf)
    tf_lists[word] = word_tf 
    
list(tf_lists.items())[:5]    

[('einstein',
  [0.2,
   0.2,
   0.1,
   0.3333333333333333,
   0.1111111111111111,
   0.08333333333333333,
   0.3333333333333333,
   0.058823529411764705,
   0.5,
   0.09090909090909091,
   0.2,
   0.125,
   0.2,
   0.5,
   0.2,
   0.25,
   0.09090909090909091,
   0.1,
   0.3333333333333333,
   0.3333333333333333,
   0.07692307692307693,
   0.3333333333333333,
   0.5,
   0.1111111111111111,
   0.2,
   0.2,
   0.3333333333333333,
   0.125,
   0.041666666666666664,
   0.05,
   0.2,
   0.14285714285714285,
   0.06666666666666667,
   0.07142857142857142,
   0.1111111111111111,
   0.3333333333333333,
   0.2,
   0.14285714285714285,
   0.125,
   0.14285714285714285,
   0.2,
   0.06666666666666667,
   0.3333333333333333,
   0.08333333333333333,
   0.0625,
   0.16666666666666666,
   0.1111111111111111,
   0.25,
   0.2,
   0.2,
   0.1111111111111111,
   0.2,
   0.2,
   0.14285714285714285,
   0.2,
   0.3333333333333333,
   0.16666666666666666,
   0.14285714285714285,
   0.07142857142857142,
  

In [66]:
zipped = list(zip(Doc_ids.values(), tf_lists.values()))
double_zipped = []
for alist in zipped:
    zippit = list(zip(alist[0],alist[1]))
    double_zipped.append(zippit)

In [177]:
inverted_file = {}
for i in range(len(q2vocab)):
    inverted_file[q2vocab[i]] = double_zipped[i]
    
list(inverted_file.items())[:5]    

[('einstein',
  [[526339.0, 2.4185433543695307],
   [50180.0, 2.4185433543695307],
   [18447.0, 1.2092716771847654],
   [35857.0, 4.030905590615884],
   [254996.0, 1.3436351968719613],
   [370203.0, 1.007726397653971],
   [367646.0, 4.030905590615884],
   [225826.0, 0.7113362806969208],
   [60963.0, 6.046358385923827],
   [369191.0, 1.0993378883497866],
   [422954.0, 2.4185433543695307],
   [428076.0, 1.5115895964809567],
   [459318.0, 2.4185433543695307],
   [464441.0, 6.046358385923827],
   [326714.0, 2.4185433543695307],
   [503359.0, 3.0231791929619134],
   [453703.0, 1.0993378883497866],
   [52295.0, 1.2092716771847654],
   [98378.0, 4.030905590615884],
   [173135.0, 4.030905590615884],
   [100944.0, 0.9302089824498195],
   [179280.0, 4.030905590615884],
   [381527.0, 6.046358385923827],
   [21594.0, 1.3436351968719613],
   [521324.0, 2.4185433543695307],
   [143474.0, 2.4185433543695307],
   [198774.0, 4.030905590615884],
   [165504.0, 1.5115895964809567],
   [240786.0, 0.5038631

In [178]:
list(inverted_file[q2vocab[0]][0])[1]

2.4185433543695307

In [179]:
for word in inverted_file:
    for tup in range(len(inverted_file[word])):
        inverted_file[word][tup] = list(inverted_file[word][tup])
        tf_idf = inverted_file[word][tup][1] * np.log10(IDF[word])
        inverted_file[word][tup][1] = tf_idf
list(inverted_file.items())[1]    

('header',
 [[294790.0, 7.516797363292878],
  [118919.0, 5.637598022469659],
  [496268.0, 9.664453752805128],
  [262797.0, 13.53023525392718],
  [265618.0, 13.53023525392718],
  [301600.0, 11.275196044939317],
  [175392.0, 6.150106933603264],
  [49071.0, 7.516797363292878],
  [382386.0, 13.53023525392718],
  [223288.0, 16.912794067408978],
  [388930.0, 11.275196044939317],
  [485316.0, 9.664453752805128],
  [537796.0, 6.76511762696359],
  [523335.0, 16.912794067408978],
  [351048.0, 9.664453752805128],
  [27092.0, 16.912794067408978],
  [27093.0, 6.76511762696359],
  [459614.0, 11.275196044939317],
  [404325.0, 13.53023525392718],
  [135655.0, 8.456397033704489],
  [349674.0, 5.203936636125839],
  [324473.0, 33.825588134817956],
  [498171.0, 13.53023525392718]])

In [180]:
for i in range(len(inverted_file)):
    inverted_file[q2vocab[i]] = [dict([[x[0],x[1]]])
                                for x in inverted_file[q2vocab[i]]]
    